In [2]:
! pip install transformers

  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 6.5 MB/s eta 0:00:00 0:00:01
Using cached regex-2024.11.6-cp311-cp311-macosx_11_0_arm64.whl (284 kB)
Using cached safetensors-0.4.5-cp311-cp311-macosx_11_0_arm64.whl (381 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 6.5 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)


In [4]:
! pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 6.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 5.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from transformers import BertTokenizer, BertModel, pipeline
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re

/Users/daianaszymczyk/Desktop/neuefische/Capstone-Project-Recommender-System/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import data

In [2]:
file_path = r'user_clean_data_ecommerce.csv'  # Update this with your file path
df_user = pd.read_csv(file_path)

In [3]:
df_user['cleaned_text'][0]

'this spray is really nice. it smells really good, goes on really fine, and does the trick. i will say it feels like you need a lot of it though to get the texture i want. i have a lot of hair, medium thickness. i am comparing to other brands with yucky chemicals so im gonna stick with this. try it!'

In [4]:
df_user.head(5)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,year,cleaned_text,review_length
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,1,2020,this spray is really nice. it smells really go...,300
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,1,2020,"this product does what i need it to do, i just...",235
2,5,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,1,2020,"smells good, feels great!",25
3,1,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2022-01-28 18:13:50.220,0,1,2022,felt synthetic,14
4,5,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-30 10:02:43.534,0,1,2020,love it,7


In [5]:
df_user.dropna(inplace=True)
df_user.shape

(692536, 13)

In [6]:
df_user['text_length'] = df_user['cleaned_text'].apply(len)  # Create a new column with text lengths

# Find the max and average text length
max_length = df_user['text_length'].max()
average_length = df_user['text_length'].mean()

print(f'Max text length (characters): {max_length}')
print(f'Average text length (characters): {average_length}')

Max text length (characters): 14692
Average text length (characters): 170.48680357411024


Split the dataset

In [7]:
train_data, test_data = train_test_split(df_user, test_size=0.2, random_state=42)

In [8]:
print(train_data.shape)
print(test_data.shape)

(554028, 14)
(138508, 14)


Create the embeddings for the first 50 000 rows of the training dataset:

In [9]:
train_subset_data_01 = train_data[:50000].copy()

In [10]:
train_subset_data_unique_users = train_subset_data_01.user_ID.nunique()
train_subset_data_unique_users

49334

### Embeddings with BERT

* BertTokenizer:
This is a tokenizer class from the transformers library provided by Hugging Face. It converts raw text into token IDs that a BERT model can process.
BERT base uncased:
"Base": Refers to the smaller version of BERT with 12 layers, 768 hidden units, and 110 million parameters.

In [11]:
# Check if a CUDA-compatible GPU is available on your system. "cuda:0" refers to the first GPU on the system (if there are multiple GPUs). If no GPU is available, device defaults to "cpu".
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [12]:
# Move the model to the GPU
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### Old funciton for creating the embeddings:

In [58]:
# # Function to get BERT embeddings
# def get_bert_embeddings(text):
#     # Tokenize input text
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512) # for now I'm gonna keep 512 (so it is max that BERT can take, because some sentences are very long)
    
#     # Move input tensors to the same device as the model (GPU or CPU)
#     inputs = {key: value.to(device) for key, value in inputs.items()}
    
#     # Forward pass through the model
#     outputs = model(**inputs)
    
#     # Get the embeddings (mean of the last hidden state)
#     return outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()  # Move the result back to CPU if needed

### Updated function by chunking for long text:

In [13]:
# Updated function to get BERT embeddings with chunking
def get_bert_embeddings(text):
    """
    Generates BERT embeddings for a given text, with support for chunking if the text is longer than 512 tokens.
    """
    # Tokenize the input text into subwords
    tokens = tokenizer.tokenize(text)  # Tokenize into subwords
    max_length = 512  # Maximum token length BERT can handle
    overlap = 50      # Overlap between chunks for better context continuity

    # If the text is short, process it directly
    if len(tokens) <= max_length:
        # Tokenize, truncate, and pad
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=max_length)
        
        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Forward pass through the model
        outputs = model(**inputs)
        
        # Get the embeddings (mean of the last hidden state)
        return outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()

    # For long texts, split into chunks
    chunks = []
    for i in range(0, len(tokens), max_length - overlap):
        # Create a chunk of tokens with overlap
        chunk = tokens[i:i + max_length]
        # Convert tokens back to string format
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        
        # Tokenize, truncate, and pad the chunk
        inputs = tokenizer(chunk_text, return_tensors='pt', truncation=True, padding=True, max_length=max_length)
        
        # Move inputs to the same device as the model
        inputs = {key: value.to(device) for key, value in inputs.items()}
        
        # Forward pass through the model
        outputs = model(**inputs)
        
        # Get the embeddings (mean of the last hidden state for this chunk)
        chunk_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().cpu().numpy()
        chunks.append(chunk_embedding)
    
    # Aggregate chunk embeddings (mean pooling across all chunks)
    final_embedding = sum(chunks) / len(chunks)  # Average embeddings across chunks
    return final_embedding

In [14]:
print(torch.cuda.is_available())

False


**Fixing Steps to gain GPU Availability for MAC specific**

In [ ]:
import torch
print(torch.backends.mps.is_available())  # Should return True if MPS is available


In [ ]:
import torch

# Check if MPS is available, and set the device
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

print(f"Using device: {device}")


In [ ]:
# Move inputs to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}


**Verify MPS Setup**

In [ ]:
import torch
print(torch.__version__)


In [ ]:
import torch
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())  # To confirm PyTorch is built with MPS support


### Creating embeddings with BERT - run for the selected data 

* Generating embeddings for the user's review with BERT,
* From each cell with review text data, a vector of 768 dimensions will be generated and stored in a cell, in a column called 'embeddings',
* Generated embeddings will be transformed from the lists of dimenstions into separated columns (one column per dimension) - therefore 768 columns will be created.

In [15]:
train_subset_data_01['embeddings'] = train_subset_data_01['cleaned_text'].apply(get_bert_embeddings)

# test_data['embeddings'] = test_data['cleaned_text'].apply(get_bert_embeddings)

In [16]:
# Verify if the column with 'embeddings' was created:
train_subset_data_01.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,year,cleaned_text,review_length,text_length,embeddings
682279,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,2015,i only got 19 and 1 was missing a jewel and i ...,342,339,"[-0.22617722, -0.011094696, 0.4226978, 0.33175..."
433103,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,2015,really great blush for the price.,33,33,"[0.26384017, -0.3625385, 0.21874148, 0.1109524..."


In [17]:
# From the lists of embeddings generate an array:
train_subset_data_stack = np.stack(train_subset_data_01['embeddings'].values)

In [18]:
train_subset_data_stack

array([[-0.22617722, -0.0110947 ,  0.4226978 , ..., -0.13620701,
         0.25588068, -0.05269833],
       [ 0.26384017, -0.3625385 ,  0.21874148, ..., -0.18521658,
         0.11775395, -0.35469234],
       [ 0.3103492 , -0.13355128,  0.09364089, ..., -0.10025669,
        -0.24313334, -0.02831975],
       ...,
       [-0.23532064, -0.4915613 ,  0.2965917 , ..., -0.29976633,
        -0.1906004 , -0.15840675],
       [-0.16455336, -0.16273713, -0.2721616 , ...,  0.10194191,
         0.01408439,  0.19025685],
       [ 0.12986025, -0.23398387,  0.49393576, ..., -0.3435306 ,
        -0.30794126,  0.09491438]], dtype=float32)

In [19]:
# Restrict to the first 50,000 rows:
train_data_50000 = train_data.iloc[:50000].copy()

In [20]:
# Create column names for embedding dimensions:
num_dimensions = train_subset_data_stack.shape[1]
embedding_columns_names = [f'dim_{i}' for i in range(num_dimensions)]

In [21]:
# Create the DataFrame of embeddings where each column contain one dimension of vector:
embedding_df = pd.DataFrame(train_subset_data_stack, columns=embedding_columns_names)

In [22]:
# Add generated embeddings to our original user data frame (the dataframe has only X number of rows, due to the splitting the original dataset):
df_user_embedding_01 = pd.concat([train_data_50000.reset_index(drop=True), embedding_df], axis=1)

In [23]:
df_user_embedding_01.shape

(50000, 782)

In [24]:
df_user_embedding_01.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_758,dim_759,dim_760,dim_761,dim_762,dim_763,dim_764,dim_765,dim_766,dim_767
0,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,...,-0.130055,-0.093996,0.183146,-0.123055,-0.168096,0.001715,-0.028300,-0.136207,0.255881,-0.052698
1,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,...,-0.115741,-0.310989,-0.029846,-0.399967,0.149936,0.052016,-0.098666,-0.185217,0.117754,-0.354692


In [25]:
# Save the original data frame with generated embeddings (new version with chunks):
df_user_embedding_01.to_csv(r'.\..\data\embeddings_output\df_user_embedding_fullsize_01_ch.csv', index=False)
# df_user_embedding_01.to_csv(r'.\..\data\embeddings_output\df_user_embedding_fullsize_01.csv', index=False)

## Next steps - dimentionality reduction with PCA

* Similar steps but here the output is a final user dataframe with vectors of 300 dimensions (In the beginning I have chosen 10 only because that's what they have done in the artile but the variance dropped to 45%, so I set up for now 300 which gives ~95%.) We can always adjust it

In [23]:
# load the data - if not run just now:

# train_subset_data_ = pd.read_csv(r'.\..\data\embeddings_output\train_subset_embeddings_02.csv') #converters={"embeddings": str})

In [26]:
train_subset_data_01.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,year,cleaned_text,review_length,text_length,embeddings
682279,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,2015,i only got 19 and 1 was missing a jewel and i ...,342,339,"[-0.22617722, -0.011094696, 0.4226978, 0.33175..."
433103,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,2015,really great blush for the price.,33,33,"[0.26384017, -0.3625385, 0.21874148, 0.1109524..."


In [27]:
# pca = PCA(n_components=10) # I have tried with 10 as they speficied in the article but the variance dropped significantly to 45%. 
pca = PCA(n_components=300)
# train_subset_data = np.stack(train_subset_data_01['embeddings'].values)

In [28]:
train_subset_data_stack

array([[-0.22617722, -0.0110947 ,  0.4226978 , ..., -0.13620701,
         0.25588068, -0.05269833],
       [ 0.26384017, -0.3625385 ,  0.21874148, ..., -0.18521658,
         0.11775395, -0.35469234],
       [ 0.3103492 , -0.13355128,  0.09364089, ..., -0.10025669,
        -0.24313334, -0.02831975],
       ...,
       [-0.23532064, -0.4915613 ,  0.2965917 , ..., -0.29976633,
        -0.1906004 , -0.15840675],
       [-0.16455336, -0.16273713, -0.2721616 , ...,  0.10194191,
         0.01408439,  0.19025685],
       [ 0.12986025, -0.23398387,  0.49393576, ..., -0.3435306 ,
        -0.30794126,  0.09491438]], dtype=float32)

In [29]:
train_pca_subset = pca.fit_transform(train_subset_data_stack)
# test_pca = pca.transform(test_embeddings)

In [30]:
print(train_pca_subset.shape)

(50000, 300)


In [31]:
train_pca_subset

array([[-2.0965383 ,  1.1980267 , -1.1503685 , ...,  0.13280511,
        -0.06080586, -0.11640628],
       [ 2.135054  , -1.1235071 ,  0.05501726, ...,  0.07214064,
         0.01684186,  0.02094016],
       [ 5.4947605 , -0.08924119, -0.78934044, ...,  0.09517658,
         0.2138613 ,  0.12064414],
       ...,
       [-0.48354256, -1.5402689 , -1.1604686 , ..., -0.24500464,
         0.00554556, -0.20773911],
       [ 0.57096297,  0.7796596 , -1.5284383 , ..., -0.14529619,
        -0.01152902,  0.02571426],
       [ 1.2441053 , -0.60658664, -1.25715   , ...,  0.03788924,
         0.09924201,  0.10208445]], dtype=float32)

Create a dataframe with UserID and embedding dimensions columns:

In [32]:
# Create column names for embedding dimensions
num_dimensions = train_pca_subset.shape[1]
embedding_columns_PCA = [f'dim_{i}' for i in range(num_dimensions)]

In [33]:
embedding_df_PCA = pd.DataFrame(train_pca_subset, columns=embedding_columns_PCA)

In [34]:
final_df_user_PCA = pd.concat([train_data_50000.reset_index(drop=True), embedding_df_PCA], axis=1)

In [35]:
# Save PCA version witch 'chunk' text modification (added _ch in a name):
final_df_user_PCA.to_csv(r'.\..\data\embeddings_dim_reduction\df_final_user_PCA_ch.csv', index=False)
# final_df_user_PCA.to_csv(r'.\..\data\embeddings_dim_reduction\df_final_user_PCA.csv', index=False)

In [36]:
final_df_user_PCA.head(5)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,...,-0.019821,-0.063556,0.068237,-0.003840,-0.034444,0.004510,-0.007486,0.132805,-0.060806,-0.116406
1,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,...,-0.001708,0.092145,-0.080462,-0.088962,0.036188,-0.143553,0.105333,0.072141,0.016842,0.020940
2,5,Good,Very nice,[],B094VMYQJP,B094VMYQJP,AGKNJHA77X4LVIXVP56MHF4C22SA,2021-11-11 06:12:08.453,0,1,...,-0.077378,0.053112,-0.000232,-0.059528,0.056033,-0.154944,-0.175278,0.095177,0.213861,0.120644
3,5,Great Value and Easy to Use !!,I bought these bottles to add my pre-mixed pai...,[],B0758V2K3K,B0758V2K3K,AFADUDWDAFJK7YSILOUBAHSR7I5Q,2022-03-06 21:20:40.790,0,1,...,0.019987,0.062898,-0.025420,0.015355,-0.038976,-0.001320,0.005959,0.068972,-0.113554,0.063735
4,5,Best ever,This is the best curling iron I ever had in my...,[],B001MP0QH4,B01N4885PD,AHHVPEX3YEH6Z6Q5PDIJY45SOLKQ,2019-12-06 17:24:50.705,9,1,...,0.096436,0.051102,-0.208861,-0.026976,-0.024060,-0.088746,-0.059459,-0.033652,0.008485,-0.021632


In [37]:
similarity_matrix = cosine_similarity(train_pca_subset)

In [38]:
similarity_matrix[0][1]

-0.18045554

In [39]:
similarity = cosine_similarity([train_pca_subset[12]], [train_pca_subset[10]])
print("Similarity:", similarity[0][0])

Similarity: -0.035786994


## PCA - Variance analysis

* Verify if the size of reduced embeddings is big enough to still keep the meaning of the textual data

Original embeddings:

In [40]:
user_embeddings_original = pd.read_csv(r'.\..\data\embeddings_output\df_user_embedding_fullsize_01_ch.csv')
user_embeddings_original.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_758,dim_759,dim_760,dim_761,dim_762,dim_763,dim_764,dim_765,dim_766,dim_767
0,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,...,-0.130055,-0.093996,0.183146,-0.123055,-0.168096,0.001715,-0.028300,-0.136207,0.255881,-0.052698
1,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,...,-0.115741,-0.310989,-0.029846,-0.399967,0.149936,0.052016,-0.098666,-0.185217,0.117754,-0.354692


In [41]:
user_embeddings_original.shape

(50000, 782)

In [42]:
unique_values_user_original = user_embeddings_original.user_ID.nunique()
unique_values_user_original

49334

In [43]:
original_embeddings = user_embeddings_original[[f"dim_{i}" for i in range(768)]]

In [44]:
# Fit PCA to the original embeddings
pca = PCA(n_components=300)
pca.fit(original_embeddings)

# Check explained variance
explained_variance = pca.explained_variance_ratio_
total_variance = sum(explained_variance)

print("Explained Variance Ratio for 300 Dimensions:", explained_variance)
print("Total Explained Variance Captured by 300 Dimensions:", total_variance)

Explained Variance Ratio for 300 Dimensions: [0.15770539 0.04569085 0.04178757 0.0338528  0.03102464 0.02787921
 0.02636755 0.02194665 0.02034437 0.01833851 0.01677296 0.01598295
 0.01471035 0.01364272 0.01254376 0.01161625 0.01046816 0.01018101
 0.0099389  0.00890585 0.0086756  0.0080092  0.00779689 0.00759841
 0.00722639 0.00686972 0.00682086 0.00650492 0.0063136  0.00610409
 0.00603635 0.00575361 0.00541211 0.0052985  0.00505044 0.00496439
 0.00490285 0.0046469  0.00454809 0.00427476 0.00419961 0.00404253
 0.00397964 0.00389777 0.0037465  0.00370564 0.00361358 0.00351666
 0.00347613 0.00337534 0.00336742 0.00329549 0.00314648 0.00312515
 0.00297752 0.00290767 0.00289047 0.00285901 0.00279358 0.00277922
 0.00272794 0.0026373  0.00260225 0.00255192 0.00245784 0.00243022
 0.00236718 0.0023391  0.00229463 0.00223147 0.00219138 0.00214752
 0.00213039 0.0020896  0.00206988 0.00203307 0.0020144  0.00195577
 0.00191488 0.00186871 0.00184891 0.00182126 0.00179657 0.00174935
 0.00173724 0.001

# Once all embeddings are calculated we can go to the next step (so far calculations are made only for the sample of 50000 rows of the training dataset)

## Create User and Product Vectors

* Aggregating the embeddings to compute a single user vector per user_ID and single product vector per product_ID.
* To create a vector for each user, you can average all the review embeddings corresponding to that user. This works because averaging retains a general sense of the user's overall preferences.
* Similarly, you can aggregate the embeddings for each product. For example, average all the embeddings corresponding to each product ID.

In [45]:
# load the dataset of PCA embeddings: 
final_df_user_PCA = pd.read_csv(r'.\..\data\embeddings_dim_reduction\df_final_user_PCA_ch.csv')

In [46]:
# Verify the number of unique users:
unique_values_user = final_df_user_PCA.user_ID.nunique()
unique_values_user

49334

In [47]:
final_df_user_PCA.head(5)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
0,4,good for the price,I only got 19 and 1 was missing a jewel and I ...,[],B00L62HMQG,B00L62HMQG,AF3BP6BP64XWBLQ6GTZKASW33EEQ,2015-08-05 09:04:21.000,0,1,...,-0.019821,-0.063556,0.068237,-0.003840,-0.034444,0.004510,-0.007486,0.132805,-0.060806,-0.116406
1,5,Five Stars,Really great blush for the price.,[],B00BMW24TU,B00BMW24TU,AGNRWZQ345LQCQ2M2X67JHJPK6OA,2015-08-22 01:13:49.000,0,1,...,-0.001708,0.092145,-0.080462,-0.088962,0.036188,-0.143553,0.105333,0.072141,0.016842,0.020940
2,5,Good,Very nice,[],B094VMYQJP,B094VMYQJP,AGKNJHA77X4LVIXVP56MHF4C22SA,2021-11-11 06:12:08.453,0,1,...,-0.077378,0.053112,-0.000232,-0.059528,0.056033,-0.154944,-0.175278,0.095177,0.213861,0.120644
3,5,Great Value and Easy to Use !!,I bought these bottles to add my pre-mixed pai...,[],B0758V2K3K,B0758V2K3K,AFADUDWDAFJK7YSILOUBAHSR7I5Q,2022-03-06 21:20:40.790,0,1,...,0.019987,0.062898,-0.025420,0.015355,-0.038976,-0.001320,0.005959,0.068972,-0.113554,0.063735
4,5,Best ever,This is the best curling iron I ever had in my...,[],B001MP0QH4,B01N4885PD,AHHVPEX3YEH6Z6Q5PDIJY45SOLKQ,2019-12-06 17:24:50.705,9,1,...,0.096436,0.051102,-0.208861,-0.026976,-0.024060,-0.088746,-0.059459,-0.033652,0.008485,-0.021632


Create a dataframe that stores all items that the user purchased, to remove those from the recommendations

In [48]:
user_item_df = final_df_user_PCA.groupby('user_ID')['product_ID'].apply(list).reset_index()
user_item_df

,user_ID,product_ID
0,AE222BBOVZIF42YOOPNBXL4UUMYA,[B013HR1A92]
1,AE223UUOHC3V2XF4JOTTDDSBODSQ,[B06Y5Y3R5L]
2,AE225T2ZALMW2LGAWZGONMYD2VJA,[B09Y1BWYCR]
3,AE225THXPLSO5QQ3PHQP7SPETM2A,[B07RVPC6X5]
4,AE226AADLLWDPLLWOEGKUCBNZH4A,[B007MAJFD4]
...,...,...
49329,AHZZOHEFU5EO466C2BAPPVV7CEJA,[B08PVXSM6Z]
49330,AHZZOMWUCOOYQWZUIK4QQPKQE46Q,[B08KHCMN8X]
49331,AHZZPMMB4Y5DHXAXOTBFALIKPU7A,[B09GFTS277]
49332,AHZZSYRRKQ4QAOTMYS5RHYWJU4HA,[B00BLWWY7S]


### 1. Aggregate User Vectors

In [49]:
# Select relevant columns: user_ID and embedding dimensions
user_vector_cols = [f'dim_{i}' for i in range(300)]
df_user_vectors = final_df_user_PCA.groupby('user_ID')[user_vector_cols].mean().reset_index()

# Rename the columns to indicate these are user vectors:
df_user_vectors.columns = ['user_ID'] + [f'user_vec_{i}' for i in range(300)]

df_user_vectors.shape

(49334, 301)

In [50]:
df_user_vectors.head()

,user_ID,user_vec_0,user_vec_1,user_vec_2,user_vec_3,user_vec_4,user_vec_5,user_vec_6,user_vec_7,user_vec_8,...,user_vec_290,user_vec_291,user_vec_292,user_vec_293,user_vec_294,user_vec_295,user_vec_296,user_vec_297,user_vec_298,user_vec_299
0,AE222BBOVZIF42YOOPNBXL4UUMYA,1.193941,-0.031830,-0.546001,1.647317,-1.807711,0.085619,-1.147377,0.331448,1.707215,...,0.097558,0.176352,-0.083327,0.165161,-0.110643,0.046314,0.067980,-0.006054,0.102686,-0.131911
1,AE223UUOHC3V2XF4JOTTDDSBODSQ,7.270739,0.962163,0.227081,-1.170759,0.693374,-2.156540,-0.341290,-0.047270,0.562144,...,-0.057659,0.081625,-0.034728,0.050262,0.083074,-0.064798,-0.053431,-0.004940,-0.081213,0.098828
2,AE225T2ZALMW2LGAWZGONMYD2VJA,0.174957,-0.948434,1.882204,-0.447084,-0.092243,0.112224,-0.193617,-0.125194,0.054014,...,0.156546,0.137741,0.125628,0.011926,-0.112042,0.128980,0.141456,0.015066,-0.199265,0.112540
3,AE225THXPLSO5QQ3PHQP7SPETM2A,-1.935537,0.112037,-0.131345,0.424575,0.312248,-0.921696,-0.190516,-0.187696,0.513130,...,-0.011821,-0.061849,0.071173,-0.001506,0.057282,-0.066050,0.032554,0.057632,-0.023802,0.002494
4,AE226AADLLWDPLLWOEGKUCBNZH4A,-1.642186,0.869895,0.106129,-0.015726,-0.932361,-0.748167,-0.607926,0.023840,-0.454447,...,-0.053200,-0.020440,-0.134453,-0.102092,-0.012563,-0.006425,0.082789,-0.022207,0.051556,-0.028021


In [51]:
# Save the vectors
df_user_vectors.to_csv(r'.\..\data\text_analysis\user_vectors\user_vectors_ch.csv', index=False)

### 2. Aggregate Product Vectors

In [52]:
product_vector_cols = [f'dim_{i}' for i in range(300)]
df_product_vectors = final_df_user_PCA.groupby('product_ID')[product_vector_cols].mean().reset_index()

df_product_vectors.columns = ['product_ID'] + [f'product_vec_{i}' for i in range(300)]
df_product_vectors.shape

(25915, 301)

In [53]:
df_product_vectors.head()

,product_ID,product_vec_0,product_vec_1,product_vec_2,product_vec_3,product_vec_4,product_vec_5,product_vec_6,product_vec_7,product_vec_8,...,product_vec_290,product_vec_291,product_vec_292,product_vec_293,product_vec_294,product_vec_295,product_vec_296,product_vec_297,product_vec_298,product_vec_299
0,0005946468,6.730884,0.135069,1.123494,-1.279462,0.912437,-1.771424,-0.112067,0.214277,0.129672,...,0.026336,0.007485,0.059004,-0.040828,-0.043456,-0.015900,0.053977,-0.043104,0.010400,-0.017636
1,069267599X,-0.525599,0.345587,1.282990,-0.358171,-0.505466,-0.233775,-0.778401,0.085403,-0.501697,...,0.064897,-0.016684,0.007002,-0.054684,0.000754,0.094406,0.016628,-0.022596,0.019918,-0.039174
2,0816091846,0.029594,0.861594,0.458658,-0.143015,0.075876,-0.613350,-1.103844,-0.114897,-0.403521,...,0.003169,-0.039732,0.066379,0.062134,-0.165674,0.094985,-0.076364,0.116423,0.021354,0.134106
3,0977217213,3.834752,0.291673,-0.880156,-0.309400,0.787636,-0.309845,0.748970,-0.470043,0.135278,...,0.097169,0.147651,0.041311,-0.313500,-0.369855,-0.137865,-0.209576,-0.220153,-0.051437,0.225075
4,1421790432,-1.453547,0.149348,1.918756,-0.578314,-0.207080,-0.307164,-1.363810,-0.655062,-0.021825,...,-0.128674,-0.161629,-0.161663,0.059190,-0.145624,0.049629,0.000237,0.030797,-0.056103,0.136859


In [54]:
# Save product vectors
df_product_vectors.to_csv(r'.\..\data\text_analysis\product_vectors\product_vectors_ch.csv', index=False)

# Semantic Analysis

## Compare Reviews - Similarity Research with COS Similarity

* Extract the user and product vectors.
* Compute the cosine similarity between each user and all products.
* Rank products for each user based on similarity scores.

In [55]:
# Extract user vectors
user_ids = df_user_vectors['user_ID'].values
user_vectors = df_user_vectors.iloc[:, 1:].values

# Extract product vectors
product_ids = df_product_vectors['product_ID'].values
product_vectors = df_product_vectors.iloc[:, 1:].values

In [56]:
print(f"Shape of User Vectors: {user_vectors.shape}")
print(f"Shape of Product Vectors: {product_vectors.shape}")

Shape of User Vectors: (49334, 300)
Shape of Product Vectors: (25915, 300)


In [57]:
# Compute cosine similarity between all users and all products
cosine_similarities = cosine_similarity(user_vectors, product_vectors)

print(f"Cosine Similarity Matrix Shape: {cosine_similarities.shape}")
# Rows correspond to users, columns correspond to products

Cosine Similarity Matrix Shape: (49334, 25915)


In [58]:
def recommend_top_n_products_by_user_id(user_id, user_ids, similarity_matrix, product_ids, top_n=5):
    """
    Recommend top N products for a given user based on cosine similarity.

    Parameters:
    - user_index: Index of the user in the similarity matrix
    - similarity_matrix: Cosine similarity matrix (users x products)
    - product_ids: List of product IDs corresponding to columns in similarity matrix
    - top_n: Number of top recommendations to return

    Returns:
    - List of (product_id, similarity_score) tuples
    """

    # Find the index of the user_ID
    if user_id not in user_ids:
        raise ValueError(f"User ID '{user_id}' not found in the user data.")
    
    user_index = np.where(user_ids == user_id)[0][0]  # Locate the index of user_id
    
    # Get similarity scores for the user
    user_similarities = similarity_matrix[user_index]
    
    # Step 4: Retrieve the list of already purchased products for the user
    purchased_products = user_item_df.loc[user_item_df['user_ID'] == user_id, 'product_ID']
    if not purchased_products.empty:
        purchased_products = set(purchased_products.iloc[0])  # Convert to a set for faster lookups
    else:
        purchased_products = set()

    # Step 5: Sort product indices by similarity scores in descending order
    sorted_indices = user_similarities.argsort()[::-1]  # Sort in descending order
    
    # Step 6: Filter out already purchased products
    recommendations = []
    for i in sorted_indices:
        product = product_ids[i]
        if product not in purchased_products:
            recommendations.append((product, user_similarities[i]))
        if len(recommendations) >= top_n:  # Stop when we have enough recommendations
            break
    
    return recommendations
# Example usage: Get recommendations for a specific user_ID
user_id_input = "AGGS22XG63AQTYBR3APBQA25HAYQ"  # Replace with any valid user_ID
top_n = 5

try:
    recommendations = recommend_top_n_products_by_user_id(user_id_input, user_ids, cosine_similarities, product_ids, top_n)
    print(f"Top-{top_n} Recommendations for User '{user_id_input}':")
    for product_id, score in recommendations:
        print(f"Product ID: {product_id}, Similarity Score: {score:.4f}")
except ValueError as e:
    print(e)

Top-5 Recommendations for User 'AGGS22XG63AQTYBR3APBQA25HAYQ':
Product ID: B0B8Q3WTGP, Similarity Score: 0.6705
Product ID: B01N41MYDC, Similarity Score: 0.6373
Product ID: B08GWZKPPL, Similarity Score: 0.6264
Product ID: B00XFGNPE4, Similarity Score: 0.6255
Product ID: B00069Q306, Similarity Score: 0.6235


In [59]:
duplicate_user_ids = final_df_user_PCA['user_ID'][final_df_user_PCA['user_ID'].duplicated()].unique()
duplicate_user_ids

array(['AEZP6Z2C5AVQDZAJECQYZWQRNG3Q', 'AGGS22XG63AQTYBR3APBQA25HAYQ',
       'AERDBP26RSGAGWWORPE6MQADNR2Q', 'AHVQR7U4PJ7VTG36DQMRSIHK7O6A',
       'AFWVN52MRBWOTIK7UGXBWGOY4HBA', 'AHGQDSOBAIVAAKJFIVEGS344MSXA',
       'AFPBV45MYM2HLBT2AH2JJ3FUADUQ', 'AHY2TURQPNIDXZGH2CMQLZ343YMQ',
       'AHYOSWORVZFXM5QMRIAW3JTTFFIQ', 'AE5UWYX65MMYGGAHNP2UJONMRRXQ',
       'AGI627XXI5DPCMEHBJKZU7WPQK6Q', 'AECQQBG6YRYCOJL2NCB2H3V6LD6Q',
       'AE5IMGWRBJA7JQFBQTBK25HDYGVA', 'AG7JCEMC64AM7JPATDVGP6YZOTXA',
       'AH6YII7DVISC5SBK62F5POULSI6A', 'AGOYO44Q3YUXRQ5N2YZRVSYTYEWQ',
       'AFRJTKBKVBAHKPIV3OIYAGS6AFRQ', 'AGZUJTI7A3JFKB4FP5JOH6NVAJIQ_1',
       'AG6PJGM3ZTNOH6RY4G6GKPCMHT5A', 'AGFTTUBGWZRDVULR3NRZYQNS5IHQ',
       'AF3XZ6UDOR4V6SMVUKWQ3IZ3JO5Q', 'AEUEEZXVXRWGO5LWPIS25R2LCKKA',
       'AG73BVBKUOH22USSFJA5ZWL7AKXA', 'AGMSR3KN2YEMEKFWWXEPIZMJKYAA',
       'AGCAJ2SSP4VUHIX3CHMEGFEYPD4Q', 'AFXF3EGQTQDXMRLDWFU7UBFQZB7Q',
       'AGBLLVBF7XRK6LIZW52ATUFQBGQA', 'AEAXAJACFMXIAAH4WOHRMXPSZWFA',
    

In [60]:
df_PCA_single_user_check = final_df_user_PCA[final_df_user_PCA['user_ID']=='AGGS22XG63AQTYBR3APBQA25HAYQ']
df_PCA_single_user_check

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
175,3,"Does not hold a lot of hair, even though ...","Does not hold a lot of hair, even though the d...",[],B000X20Y4C,B000X20Y4C,AGGS22XG63AQTYBR3APBQA25HAYQ,2016-04-29 02:18:14.000,2,1,...,0.038800,0.017651,-0.036641,-0.058357,0.038531,-0.048998,-0.010497,-0.104376,0.034461,0.005634
2347,5,Love it,When the item arrived to my house and I was ab...,[],B00DPNKDR4,B0CDNZ7F2V,AGGS22XG63AQTYBR3APBQA25HAYQ,2016-04-29 00:54:59.000,8,1,...,-0.005958,-0.046481,-0.034873,0.021876,0.058862,-0.018914,-0.036426,0.035241,-0.124052,-0.092053


Analyse recommended product

In [61]:
product_ids_to_filter = ['B00XFGNPE4', 'B01N41MYDC','B08GWZKPPL','B0B8Q3WTGP','B00069Q306']  # List of product IDs you want to filter
filtered_df = final_df_user_PCA[final_df_user_PCA['product_ID'].isin(product_ids_to_filter)]
filtered_df

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_290,dim_291,dim_292,dim_293,dim_294,dim_295,dim_296,dim_297,dim_298,dim_299
8375,4,Epil Stop 'n Spray -- worked for me!,"After reading all the negative reviews, I wasn...",[],B00069Q306,B00069Q306,AFIQ3YH27NICN4VXZ5C32K5S75IA,2007-05-16 19:06:02.000,5,0,...,-0.026235,0.020464,-0.093683,0.018329,-0.030364,0.038579,-0.052889,-0.041194,0.038073,0.004138
22425,5,i finally found a great non-toxic styling prod...,"I rarely use hair products, because a) I don't...",[],B00XFGNPE4,B00XFGNPE4,AFXIGVQQTHA3WEZNZZ73MNI6XZQA,2015-07-26 12:36:52.000,0,0,...,-0.015847,-0.063813,0.022161,0.018181,0.018751,0.067470,-0.045869,0.047467,-0.022225,-0.079766
39088,5,"Lightweight, Works Great!",I have naturally curly hair and blow dry it st...,[{'small_image_url': 'https://m.media-amazon.c...,B0B8Q3WTGP,B0B8Q3WTGP,AEITAF3YU6VVHFTOZZLPRFBLJE4A,2022-08-29 12:33:13.546,0,0,...,-0.020212,-0.003591,-0.083836,0.013939,0.044248,0.075536,0.015353,-0.042331,-0.013520,-0.067249
43411,2,Disappointed,I was disappointed in this pair of cuticle cli...,[],B01N41MYDC,B01N41MYDC,AGWADILSGQN3BRNWRRYCHDXTJOKQ,2017-04-26 17:38:21.000,0,1,...,-0.040849,-0.055261,-0.009836,0.155980,-0.000776,0.056130,0.106755,-0.056152,-0.023497,-0.104203
49737,3,Horrible chemical smell when it heats,What I hate most about certain products is tha...,[],B08GWZKPPL,B08GWZKPPL,AFWUV4IQC7OFBDAXTPNIKGX5VXDA,2020-12-11 02:20:18.437,0,1,...,-0.040779,-0.015281,-0.008701,0.017587,-0.040965,0.016921,0.006198,-0.003827,-0.007228,-0.035167


In [62]:
pd.set_option('display.max_colwidth', None)

# Display the cleaned_text column
print(filtered_df['cleaned_text'])

8375                                                          after reading all the negative reviews, i wasnt expecting epil stop n spray to be very effective. boy, was i surprised.i did not have to spray it on multiple times to remove hair. one layer worked fine. i let it set in for about 10 minutes, applied a reasonable amount of pressure during the removal process and all the hair came off!many people have reported that it burned or irritated their skin. not so in my case. the only complaint i have is that the spray really does smell. but so does nair. no big deal.this actually worked better than nair and most of the hair removal products ive used.i dont know how effective the rollon or wipes are, but after seeing how well the spray worked, i just might try those too.
22425                                                                                                                                                       i rarely use hair products, because a i dont know how to use t

# Sentiment Analysis

* To classify customer reviews into sentiment categories, such as positive, neutral, or negative.
* Fine-tune BERT (Optional):

Train a BERT-based model on your labeled sentiment data (e.g., reviews with labels like "positive," "negative").
This improves sentiment prediction accuracy for domain-specific language.

* Direct Embedding-Based Classification:

Alternatively, use pre-trained embeddings and train a simple classifier (e.g., logistic regression) on top.

* Output:

Predict the sentiment for each review and quantify overall customer satisfaction.

**Step-by-Step Process for Sentiment Analysis**

    Install additional libraries: You'll need libraries such as nltk, TextBlob, or transformers for sentiment analysis. Below I'll demonstrate using TextBlob for simplicity, as it is easy to use and provides a quick way to analyze sentiment. Alternatively, you could use a model from Hugging Face's transformers library for a more advanced approach.

    Load and preprocess the text: You've already cleaned the text in the cleaned_text column, so there’s no further preprocessing required. However, ensure the text is in a proper format for sentiment analysis.

    Sentiment analysis using TextBlob: TextBlob provides a polarity score for sentiment analysis, where:

    Positive polarity: > 0
    Negative polarity: < 0
    Neutral polarity: = 0

    Store results: After performing the analysis, you can add a new column to your dataframe that contains the sentiment scores and labels (positive, negative, or neutral).


In [64]:
# Step 1: Install necessary libraries (if not already installed)
!pip install textblob

from textblob import TextBlob

# Step 2: Define a function for sentiment analysis
def analyze_sentiment(text):
    # Create a TextBlob object
    blob = TextBlob(text)
    
    # Get the polarity of the text
    polarity = blob.sentiment.polarity
    
    # Define the sentiment based on polarity
    if polarity > 0:
        sentiment = 'positive'
    elif polarity < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    return polarity, sentiment

# Step 3: Apply sentiment analysis to the 'cleaned_text' column
filtered_df[['sentiment_polarity', 'sentiment']] = filtered_df['cleaned_text'].apply(analyze_sentiment).apply(pd.Series)

# Step 4: Display the results
print(filtered_df[['cleaned_text', 'sentiment_polarity', 'sentiment']])

# Optional: Save the results to a CSV file if needed
# filtered_df.to_csv('sentiment_analysis_results.csv', index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.3 MB/s eta 0:00:00a 0:00:01
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               cleaned_text   
8375                                                

/var/folders/j0/trfyn8c941s6m4064vpz_1rw0000gn/T/ipykernel_5231/1932872301.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['sentiment_polarity', 'sentiment']] = filtered_df['cleaned_text'].apply(analyze_sentiment).apply(pd.Series)
/var/folders/j0/trfyn8c941s6m4064vpz_1rw0000gn/T/ipykernel_5231/1932872301.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[['sentiment_polarity', 'sentiment']] = filtered_df['cleaned_text'].apply(analyze_sentiment).apply(pd.Series)


**Advanced Sentiment Analysis using Hugging Face's pre-trained transformers model**

In [65]:
from transformers import pipeline

# Load pre-trained sentiment analysis model from Hugging Face
sentiment_analyzer = pipeline("sentiment-analysis")

# Apply the sentiment analysis
filtered_df['advanced_sentiment'] = filtered_df['cleaned_text'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

# Display the results
print(filtered_df[['cleaned_text', 'advanced_sentiment']])


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               cleaned_text   
8375                                                        after reading all the negative reviews, i wasnt expecting epil stop n spray to be very effective. boy, was i surprised.i did not have to spray it on multiple

/var/folders/j0/trfyn8c941s6m4064vpz_1rw0000gn/T/ipykernel_5231/1325502416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['advanced_sentiment'] = filtered_df['cleaned_text'].apply(lambda x: sentiment_analyzer(x)[0]['label'])
